In [2]:
#importing libraries
import sqlite3 as sql
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [3]:
#reading csv file into dataframe
cust_account = pd.read_csv('cust_account.csv')
cust_account.head()

,Account_id,Tenure,Contract,PaymentMethod,PaperlessBilling,MonthlyCharges,TotalCharges
0,3668-QPYBK,2,Month-to-month,Mailed check,Yes,53.85,108.15
1,9237-HQITU,2,Month-to-month,Electronic check,Yes,70.70,151.65
2,9305-CDSKC,8,Month-to-month,Electronic check,Yes,99.65,820.5
3,7892-POOKP,28,Month-to-month,Electronic check,Yes,104.80,3046.05
4,0280-XJGEX,49,Month-to-month,Bank transfer (automatic),Yes,103.70,5036.3


In [4]:
cust_services = pd.read_csv('cust_services.csv')
cust_services.head()

,Cust_ID,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,3668-QPYBK,Yes,No,DSL,Yes,Yes,No,No,No,No
1,9237-HQITU,Yes,No,Fiber optic,No,No,No,No,No,No
2,9305-CDSKC,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes
3,7892-POOKP,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes
4,0280-XJGEX,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes


In [5]:
customer = pd.read_csv('customer.csv')
customer.head()

,CustomerID,Gender,SeniorCitizen,Partner,Dependents
0,3668-QPYBK,Male,No,No,No
1,9237-HQITU,Female,No,No,Yes
2,9305-CDSKC,Female,No,No,Yes
3,7892-POOKP,Female,No,Yes,Yes
4,0280-XJGEX,Male,No,No,Yes


In [6]:
cust_loc = pd.read_csv('cust_loc.csv')
cust_loc.head()

,Cust_ID,State,Latitude,Longitude,ZipCode
0,3668-QPYBK,California,33.964131,-118.272783,90003
1,9237-HQITU,California,34.059281,-118.307420,90005
2,9305-CDSKC,California,34.048013,-118.293953,90006
3,7892-POOKP,California,34.062125,-118.315709,90010
4,0280-XJGEX,California,34.039224,-118.266293,90015


In [7]:
cust_churn = pd.read_csv('cust_churn.csv')
cust_churn.head()

,Id,Churn
0,3668-QPYBK,Yes
1,9237-HQITU,Yes
2,9305-CDSKC,Yes
3,7892-POOKP,Yes
4,0280-XJGEX,Yes


In [9]:
#craeting database connection and cursor
conn = sql.connect('churn.db')
cur = conn.cursor()

In [10]:
#creating tables in database from pandas dataframe
#create cust_account table
cust_account.to_sql('cust_account', conn, if_exists='replace', index=False)

7043

In [11]:
#create cust_services table
cust_services.to_sql('cust_services', conn, if_exists='replace', index=False)

7043

In [12]:
#create cust_loc table
cust_loc.to_sql('cust_loc', conn, if_exists='replace', index=False)

7043

In [13]:
#create customer table
customer.to_sql('customer', conn, if_exists='replace', index=False)

7043

In [14]:
#create churn table
cust_churn.to_sql('cust_churn', conn, if_exists='replace', index=False)

7043

In [20]:
# function to find names of all tables and their corresponding column names
def table_info(conn, cur):
    tables = cur.execute("select name from sqlite_master where type='table';").fetchall()
    for table_name in tables:
        table_name = table_name[0]
        table = pd.read_sql_query("select * from {}".format(table_name), conn)
        print(table_name)
        for col in table.columns:
            print('\t'+ col)
            print()

In [21]:
print(table_info(conn, cur))

cust_account
	Account_id

	Tenure

	Contract

	PaymentMethod

	PaperlessBilling

	MonthlyCharges

	TotalCharges

cust_services
	Cust_ID

	PhoneService

	MultipleLines

	InternetService

	OnlineSecurity

	OnlineBackup

	DeviceProtection

	TechSupport

	StreamingTV

	StreamingMovies

cust_loc
	Cust_ID

	State

	Latitude

	Longitude

	ZipCode

customer
	CustomerID

	Gender

	SeniorCitizen

	Partner

	Dependents

cust_churn
	Id

	Churn

None


# Joining individual dataframe into one 

In [34]:
#renaming customerid on each dataframe 
cust_loc.rename(columns={'Cust_ID': 'CustomerID'}, inplace=True)
cust_account.rename(columns={'Account_id': 'CustomerID'}, inplace=True)
cust_services.rename(columns={'Cust_ID': 'CustomerID'}, inplace=True)
cust_churn.rename(columns={'Id': 'CustomerID'}, inplace=True)


In [37]:
#concating all dataframe into one dataframe
cust_final = pd.concat([customer, cust_loc, cust_account, cust_services, cust_churn], join='inner', axis=1)
cust_final.head()

,CustomerID,Gender,SeniorCitizen,Partner,Dependents,CustomerID,State,Latitude,Longitude,ZipCode,...,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,CustomerID,Churn
0,3668-QPYBK,Male,No,No,No,3668-QPYBK,California,33.964131,-118.272783,90003,...,No,DSL,Yes,Yes,No,No,No,No,3668-QPYBK,Yes
1,9237-HQITU,Female,No,No,Yes,9237-HQITU,California,34.059281,-118.307420,90005,...,No,Fiber optic,No,No,No,No,No,No,9237-HQITU,Yes
2,9305-CDSKC,Female,No,No,Yes,9305-CDSKC,California,34.048013,-118.293953,90006,...,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,9305-CDSKC,Yes
3,7892-POOKP,Female,No,Yes,Yes,7892-POOKP,California,34.062125,-118.315709,90010,...,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,7892-POOKP,Yes
4,0280-XJGEX,Male,No,No,Yes,0280-XJGEX,California,34.039224,-118.266293,90015,...,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,0280-XJGEX,Yes


In [45]:
cust_final = cust_final.loc[:, ~cust_final.columns.duplicated()]

In [50]:
cust_final.dtypes

CustomerID           object
Gender               object
SeniorCitizen        object
Partner              object
Dependents           object
State                object
Latitude            float64
Longitude           float64
ZipCode               int64
Tenure                int64
Contract             object
PaymentMethod        object
PaperlessBilling     object
MonthlyCharges      float64
TotalCharges        float64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Churn                object
dtype: object

In [49]:
#changing datatype of totalcharges field
cust_final['TotalCharges'] = pd.to_numeric(cust_final['TotalCharges'], errors='coerce')

In [53]:
#checking for null values in dataframe
cust_final.isna().sum()

CustomerID          0
Gender              0
SeniorCitizen       0
Partner             0
Dependents          0
State               0
Latitude            0
Longitude           0
ZipCode             0
Tenure              0
Contract            0
PaymentMethod       0
PaperlessBilling    0
MonthlyCharges      0
TotalCharges        0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Churn               0
dtype: int64

In [52]:
# deleting n/a values
cust_final = cust_final.dropna()

In [55]:
#creating table in database from dataframe
cust_final.to_sql('cust_final', conn, if_exists='replace', index=False)

7032

In [56]:
#fetching table details using sql query
cur.execute("pragma table_info('cust_final')").fetchall()

[(0, 'CustomerID', 'TEXT', 0, None, 0),
 (1, 'Gender', 'TEXT', 0, None, 0),
 (2, 'SeniorCitizen', 'TEXT', 0, None, 0),
 (3, 'Partner', 'TEXT', 0, None, 0),
 (4, 'Dependents', 'TEXT', 0, None, 0),
 (5, 'State', 'TEXT', 0, None, 0),
 (6, 'Latitude', 'REAL', 0, None, 0),
 (7, 'Longitude', 'REAL', 0, None, 0),
 (8, 'ZipCode', 'INTEGER', 0, None, 0),
 (9, 'Tenure', 'INTEGER', 0, None, 0),
 (10, 'Contract', 'TEXT', 0, None, 0),
 (11, 'PaymentMethod', 'TEXT', 0, None, 0),
 (12, 'PaperlessBilling', 'TEXT', 0, None, 0),
 (13, 'MonthlyCharges', 'REAL', 0, None, 0),
 (14, 'TotalCharges', 'REAL', 0, None, 0),
 (15, 'PhoneService', 'TEXT', 0, None, 0),
 (16, 'MultipleLines', 'TEXT', 0, None, 0),
 (17, 'InternetService', 'TEXT', 0, None, 0),
 (18, 'OnlineSecurity', 'TEXT', 0, None, 0),
 (19, 'OnlineBackup', 'TEXT', 0, None, 0),
 (20, 'DeviceProtection', 'TEXT', 0, None, 0),
 (21, 'TechSupport', 'TEXT', 0, None, 0),
 (22, 'StreamingTV', 'TEXT', 0, None, 0),
 (23, 'StreamingMovies', 'TEXT', 0, None, 0

In [57]:
pd.read_sql('select * from cust_final', conn)

,CustomerID,Gender,SeniorCitizen,Partner,Dependents,State,Latitude,Longitude,ZipCode,Tenure,...,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Churn
0,3668-QPYBK,Male,No,No,No,California,33.964131,-118.272783,90003,2,...,Yes,No,DSL,Yes,Yes,No,No,No,No,Yes
1,9237-HQITU,Female,No,No,Yes,California,34.059281,-118.307420,90005,2,...,Yes,No,Fiber optic,No,No,No,No,No,No,Yes
2,9305-CDSKC,Female,No,No,Yes,California,34.048013,-118.293953,90006,8,...,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Yes
3,7892-POOKP,Female,No,Yes,Yes,California,34.062125,-118.315709,90010,28,...,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Yes
4,0280-XJGEX,Male,No,No,Yes,California,34.039224,-118.266293,90015,49,...,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7027,2569-WGERO,Female,No,No,No,California,34.341737,-116.539416,92285,72,...,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No
7028,6840-RESVB,Male,No,Yes,Yes,California,34.667815,-117.536183,92301,24,...,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,No
7029,2234-XADUH,Female,No,Yes,Yes,California,34.559882,-115.637164,92304,72,...,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,No
7030,4801-JZAZL,Female,No,Yes,Yes,California,34.167800,-116.864330,92305,11,...,No,No phone service,DSL,Yes,No,No,No,No,No,No


In [60]:
#verifying table names in database
cur.execute("select name from sqlite_master where type = 'table'").fetchall()

[('cust_account',),
 ('cust_services',),
 ('cust_loc',),
 ('customer',),
 ('cust_churn',),
 ('cust_final',)]

In [61]:
#closing database connection
conn.close()